In [ ]:
# import relevant libraries
import osmnx as ox
import folium as fl
import geopandas as gpd
import matplotlib.pyplot as plt

### 1. Extract Generators

In [ ]:
# load & combine power plant data from osmnx for SN & BB
power_stations_sachsen = ox.features.features_from_place("Sachsen, Germany", tags = {'power':['plant','generator']})
# power_stations_brandenburg = ox.features.features_from_place("Brandenburg, Germany", tags = {'power':['plant','generator']})

In [21]:
# print all variable names in power_stations_sachsen as dataframe without truncation
#pd.set_option('display.max_rows', None)
#print(power_stations_sachsen.columns.to_frame())
# keep only these columsn: generator:source, operator, power, generator:output:electricity, source,generator:method, generator:type, manufacturer:type, power_rating

power_stations_sachsen = power_stations_sachsen[['generator:source', 'operator', 'power', 'generator:output:electricity', 'source','generator:method', 'generator:type', 'manufacturer:type', 'power_rating']]
power_stations_sachsen.head()

generator:source operator      power  \
element id                                               
node    87853742              wind      NaN  generator   
        87855129              wind      NaN  generator   
        87855313              wind      NaN  generator   
        115278974             wind      NaN  generator   
        173372773            hydro      NaN  generator   

                  generator:output:electricity source  generator:method  \
element id                                                                
node    87853742                        1.8 MW    NaN      wind_turbine   
        87855129                        600 kW    NaN      wind_turbine   
        87855313                        600 kW    NaN      wind_turbine   
        115278974                      1000 kW  Yahoo      wind_turbine   
        173372773                          yes    NaN  run-of-the-river   

                    generator:type manufacturer:type power_rating  
element id                                                         
node    87853742   horizontal_axis               NaN          NaN  
        87855129   horizontal_axis               NaN          NaN  
        87855313   horizontal_axis               NaN          NaN  
        115278974  horizontal_axis               NaN          NaN  
        173372773              NaN               NaN          NaN

### 2. Extract Substations

In [ ]:
# load & combine power substation data from osmnx
substations_sachsen = ox.features.features_from_place("Sachsen, Germany", tags = {'power':'substation'})
substations_brandenburg = ox.features.features_from_place("Brandenburg, Germany", tags = {'power':'substation'})

### 3. Assign Bundesland Information

### 4. Associate Generators to Substations

### 5. Aggregate and Summarize